### Install packages and import

In [ ]:
import os # type:ignore # isort:skip # fmt:skip # noqa # nopep8
import sys # type:ignore # isort:skip # fmt:skip # noqa # nopep8
from pathlib import Path # type:ignore # isort:skip # fmt:skip # noqa # nopep8

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

if code_dir_name not in str(Path.cwd()).split('/')[-1]:
    for _ in range(5):

        parent_path = str(Path.cwd().parents[_]).split('/')[-1]

        if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

            code_dir = str(Path.cwd().parents[_])

            if code_dir is not None:
                break
else:
    code_dir = str(Path.cwd())
sys.path.append(code_dir)

# %load_ext autoreload
# %autoreload 2


### Define some functions

In [ ]:
# Function to check for pop-ups
def popup(driver):
    # Get html of page with beautifulsoup4
    html = driver.page_source
    print("Feeding html driver to BeautifulSoup.")
    soup = BeautifulSoup(html, "lxml")

    # Messaging popup
    try:
        messaging_soup = soup.find("li-icon", attrs={"type": "chevron-down-icon"})
        messaging_xpath = xpath_soup(messaging_soup)
        message_button = WebDriverWait(driver, 10).until(
            lambda driver: driver.find_element_by_xpath(messaging_soup)
        )
        if (message_button.is_displayed()) and (message_button.is_enabled()):
            print("Message popup detected.")
            message_button.click()
    except errors:

        # Accept cookies
        try:
            cookie_button = WebDriverWait(driver, 10).until(
                lambda driver: driver.find_element_by_css_selector(
                    "#artdeco-global-alert-container > div.artdeco-global-alert.artdeco-global-alert--NOTICE.artdeco-global-alert--COOKIE_CONSENT > section > div > div.artdeco-global-alert-action__wrapper > button:nth-child(2)"
                )
            )
            if (cookie_button.is_displayed()) and (cookie_button.is_enabled()):
                cookie_button.click()
        except errors:

            # Accept Alert
            try:
                click_alert = driver.switch_to.alert()
                print("Alert found.")
                click_alert.accept()
            except errors:
                pass
            except:
                try:
                    print(f"Alert caused unexpected error: {sys.exc_info()[0]}")
                except (NoSuchWindowException, urllib3.exceptions.MaxRetryError):
                    pass
            else:
                print("Alert accepted.")
                WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable(
                        (By.CSS_SELECTOR, "body > div.cta-modal.show > button")
                    )
                )
        except:
            try:
                print(f"Cookies alert caused unexpected error: {sys.exc_info()[0]}")
            except (NoSuchWindowException, urllib3.exceptions.MaxRetryError):
                pass
        else:
            print("Cookies accepted.")
            WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable(
                    (
                        By.CSS_SELECTOR,
                        "body > header > nav > section > section:nth-child(4) > form > button",
                    )
                )
            )
    except:
        try:
            print(f"Message popup caused unexpected error: {sys.exc_info()[0]}")
        except (NoSuchWindowException, urllib3.exceptions.MaxRetryError):
            pass
    else:
        print("Message popup dismissed.")
        WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable(
                (
                    By.CSS_SELECTOR,
                    "body > header > nav > section > section:nth-child(4) > form > button",
                )
            )
        )

    return


In [ ]:
# Function to sign in
def sign_in(driver, username=None, password=None):
    if username is None:
        os.environ.get('YAHOO_EMAIL')
    if password is None:
        os.environ.get('YAHOO_PASSWORD')
    print('Entering username and password.')

    try:
        username_textbox = WebDriverWait(driver, 10).until(
            lambda driver: driver.find_element_by_id("login-email")
        )
    except:
        username_textbox = WebDriverWait(driver, 10).until(
            lambda driver: driver.find_element_by_id("username")
        )
    try:
        password_textbox = WebDriverWait(driver, 10).until(
            lambda driver: driver.find_element_by_id("login-password")
        )
    except:
        password_textbox = WebDriverWait(driver, 10).until(
            lambda driver: driver.find_element_by_id("password")
        )

    username_textbox.send_keys(username)
    password_textbox.send_keys(password)
    password_textbox.send_keys(Keys.ENTER)

    # Get html of page with beautifulsoup4
    html = driver.page_source
    print("Feeding html driver to BeautifulSoup.")
    soup = BeautifulSoup(html, "lxml")
    time.sleep(2)
    print(f"Current page title: {driver.title}")


In [ ]:
# Function to find page number
def find_page_number(driver, soup, page_counter, keyword):
    # Get html of page with beautifulsoup4
    html = driver.page_source
    print("Feeding html driver to BeautifulSoup.")
    soup = BeautifulSoup(html, "lxml")

    try:
        try:
            page_footer = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located(
                    (By.CLASS_NAME, "jobs-search-two-pane__pagination")
                )
            )
        except:
            page_footer_soup = (
                soup.find("section", {"aria-label": "pagination"})
                .find("div", class_="pv5 artdeco-pagination ember-view")
                .find(
                    "ul",
                    class_="artdeco-pagination__pages artdeco-pagination__pages--number",
                )
            )
            page_footer_xpath = xpath_soup(page_footer_soup)
            page_footer = WebDriverWait(driver, 10).until(
                lambda driver: driver.find_element_by_class_name(page_footer_xpath)
            )
        if page_footer:
            page_number_length = driver.find_elements_by_class_name(
                "artdeco-pagination__pages artdeco-pagination__pages--number"
            )
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
            time.sleep(2)
            page_number_length = soup.find(
                "section", class_="jobs-search-two-pane__pagination"
            )  # .find('div', class_ = 'pv5 artdeco-pagination ember-view').find('ul', class_ = 'artdeco-pagination__pages artdeco-pagination__pages--number')
            print("page_number_length: ", page_number_length)
            page_number_length = (
                page_number_length[-1].find_element_by_class_name("button").text.strip()
            )
            if int(page_number_length) == 1:
                print(f"Only 1 page found for {str(keyword)} jobs.")
                page_number = page_number_length
                page_number = [int(s) for s in page_number.split() if s.isdigit()][0]
                print(f"Current page number: {page_number}.")
            elif int(page_number_length) != 1:
                print(
                    f"Total of {page_number_length} pages found for {str(keyword)} jobs."
                )
                try:
                    page_number = (
                        soup.find(
                            "li",
                            class_="artdeco-pagination__indicator artdeco-pagination__indicator--number active selected ember-view",
                        )
                        .find("button", {"aria-current": "true"})
                        .text.strip()
                    )
                except:
                    page_number = (
                        soup.find(
                            "li",
                            class_="artdeco-pagination__indicator artdeco-pagination__indicator--number active selected ember-view",
                        )
                        .find(
                            "button", attrs={"aria-label": re.compile("*current page")}
                        )
                        .text.strip()
                    )
                page_number = [int(s) for s in page_number.split() if s.isdigit()][0]
                print(f"Current page number: {page_number}.")

        elif not page_footer:
            page_number = page_counter
            page_number_length = 1
            print(
                f"No page number indicator found. Page number set to counter: {page_number}"
            )
    except (NoSuchElementException, TimeoutException):
        page_number = page_counter
        page_number_length = 1
        print(
            f"No page number indicator found. Page number set to counter: {page_number}"
        )
    except errors:
        page_number = page_counter
        page_number_length = -1
        print(f"Unexpected error getting page number: {sys.exc_info()[0]}.")

    return page_number_length, page_number


In [ ]:
# Function to find and click load more button
def load_more(
    driver,
    soup,
    keyword,
    keyword_file,
    site,
    main_window,
    window_before,
    counter,
    page_counter,
    page_number,
    page_number_length,
    jobs,
    jobs_count,
    df_old_jobs,
    save_path,
    json_file_name,
):
    try:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
        time.sleep(2)
        load_more_xpath = xpath_soup(
            soup.find(
                "section", class_="m3", attrs={"aria-label": "Load more results"}
            ).find("button")
        )
        load_more_button = WebDriverWait(driver, 10).until(
            lambda driver: driver.find_element_by_xpath(load_more_xpath)
        )
        if (load_more_button.is_displayed()) and (load_more_button.is_enabled()):
            try:
                load_more_button.click()
            except:
                time.sleep(2)
                load_more_button.click()
            finally:
                page_counter += 1
                page_number = page_counter
                page_number_length = page_number + 1
                print(
                    '"Load More" button found and clicked so no next button or page number.'
                )
                # Get jobs
                jobs, jobs_count, counter = get_page_jobs(
                    driver,
                    soup,
                    keyword,
                    keyword_file,
                    site,
                    main_window,
                    window_before,
                    counter,
                    page_counter,
                    page_number,
                    page_number_length,
                    jobs,
                    df_old_jobs,
                    save_path,
                    json_file_name,
                )
        elif (not load_more_button.is_displayed()) and (
            not load_more_button.is_enabled()
        ):
            page_number = page_counter
            page_number_length = 1
            print(
                'Could not get page number. No "Load More" button or next button found.'
            )
    except (NoSuchWindowException, NoSuchElementException):
        print(
            f"Could not go to next page. End of data collection for {keyword}.\nGot {int(jobs_count)} jobs."
        )


In [ ]:
# Function to get main job data
def get_main(driver, soup):

    pattern = re.compile(r"[^A-Za-z0-9, ]+")
    upper_pattern = re.compile("[A-Z][^A-Z\s]+(?:\s+\S[^A-Z\s]*)*")
    #     WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CLASS_NAME, 'jobs-search-results')))
    job_container = soup.find("div", class_="jobs-details-top-card__container")
    job_container2 = soup.find("div", id="job-details").find(
        "div", class_="jobs-description-details pt4"
    )

    try:
        job_title = (
            job_container.find(
                "a", class_="jobs-details-top-card__job-title-link ember-view"
            )
            .find("h2", class_="jobs-details-top-card__job-title t-20 t-black t-normal")
            .text.strip()
        )
    except NoSuchElementException:
        print(f"No job title found.")
        job_title = -1
    except errors:
        print(f"Unexpected error getting job title: {sys.exc_info()[0]}.")
        job_title = -1
    else:
        print("-" * 20)
        print(f"Job Title: {job_title}")
        print("-" * 20)

    try:
        job_description_list = []
        job_desc = soup.find_all("div", id="job-details")
        for job_d in job_desc:
            job_description_list.append(job_d.text.strip())
        job_description = "\n".join(job_description_list)
    except NoSuchElementException:
        print(f"No job description found.")
        job_description = -1
    except errors:
        print(f"Unexpected error getting job description: {sys.exc_info()[0]}.")
        job_description = -1
    else:
        print("-" * 20)
        print(f"Job Description: {job_description[:50]}")
        print("-" * 20)

    try:
        company_name = (
            job_container.find("div", class_="jobs-details-top-card__content-container")
            .find("a", class_=re.compile("^jobs-details-top-card__company-url*"))
            .text.strip()
        )
    except NoSuchElementException:
        print(f"No job company name found.")
        company_name = -1
    except errors:
        print(f"Unexpected error getting company name: {sys.exc_info()[0]}.")
        company_name = -1
    else:
        print("-" * 20)
        print(f"Company Name: {company_name}")
        print("-" * 20)

    try:
        location = soup.find(
            "li", class_="job-card-container__metadata-item"
        ).text.strip()
    except NoSuchElementException:
        print(f"No job company location found.")
        location = -1
    except errors:
        print(f"Unexpected error getting company location: {sys.exc_info()[0]}.")
        location = -1
    else:
        print("-" * 20)
        print(f"Company Location: {location}")
        print("-" * 20)

    try:
        try:
            industry_list = (
                job_container2.find("div", class_="jobs-box__group")
                .find("h3", text=re.compile("Industry"), attrs={"class": "t-14 t-bold"})
                .find_parent("div", class_="jobs-box__group")
                .find_all("ul", class_="jobs-box__list jobs-description-details__list")
            )
        except:
            industry_list = (
                job_container2.find(
                    "h3", text=re.compile("Industry"), attrs={"class": "t-14 t-bold"}
                )
                .find_parent("div", class_="jobs-box__group")
                .find_all("ul", class_="jobs-box__list jobs-description-details__list")
            )
        for ind in industry_list:
            ind = ind.text.strip().replace("  ", "").split("\n")
            industry = [i for i in ind if i]
    except NoSuchElementException:
        print(f"No job company industry found.")
        industry = -1
    except errors:
        print(f"Unexpected error getting industry: {sys.exc_info()[0]}.")
        industry = -1
    else:
        print("-" * 20)
        print(f"Industry: {industry}")
        print("-" * 20)

    try:
        try:
            sector_list = (
                job_container2.find("div", class_="jobs-box__group")
                .find(
                    "h3",
                    text=re.compile("Job Functions"),
                    attrs={"class": "t-14 t-bold"},
                )
                .find_parent("div", class_="jobs-box__group")
                .find_all("ul", class_="jobs-box__list jobs-description-details__list")
            )
        except:
            sector_list = (
                job_container2.find(
                    "h3",
                    text=re.compile("Job Functions"),
                    attrs={"class": "t-14 t-bold"},
                )
                .find_parent("div", class_="jobs-box__group")
                .find_all("ul", class_="jobs-box__list jobs-description-details__list")
            )
        for sec in sector_list:
            sec = sec.text.strip().replace("  ", "").split("\n")
            sector = [s for s in sec if s]
    except NoSuchElementException:
        print(f"No job sector found.")
        sector = -1
    except errors:
        print(f"Unexpected error getting sector: {sys.exc_info()[0]}.")
        sector = -1
    else:
        print("-" * 20)
        print(f"Sector: {sector}")
        print("-" * 20)

    try:
        try:
            level = (
                job_container2.find("div", class_="jobs-box__group")
                .find(
                    "h3",
                    text=re.compile("Seniority Level"),
                    attrs={"class": "t-14 t-bold"},
                )
                .find_parent("div", class_="jobs-box__group")
                .find("p", class_="t-14 mb3")
                .text.strip()
            )
        except:
            level = (
                job_container2.find(
                    "h3",
                    text=re.compile("Seniority Level"),
                    attrs={"class": "t-14 t-bold"},
                )
                .find_parent("div", class_="jobs-box__group")
                .find("p", class_="t-14 mb3")
                .text.strip()
            )
    except NoSuchElementException:
        print(f"No job seniority level found.")
        level = -1
    except errors:
        print(f"Unexpected error getting seniority level: {sys.exc_info()[0]}.")
        level = -1
    else:
        print("-" * 20)
        print(f"Seniority Level: {level}")
        print("-" * 20)

    try:
        company_url = (
            "https://www.linkedin.com/"
            + job_container.find(
                "a", class_=re.compile("^jobs-details-top-card__company-url*")
            )["href"]
        )
    except NoSuchElementException:
        print(f"No job company url found.")
        company_url = -1
    except errors:
        print(f"Unexpected error getting company url: {sys.exc_info()[0]}.")
        company_url = -1
    else:
        print("-" * 20)
        print(f"Company URL: {company_url}")
        print("-" * 20)

    try:
        try:
            emp_type = (
                job_container2.find("div", class_="jobs-box__group")
                .find(
                    "h3",
                    text=re.compile("Employment Type"),
                    attrs={"class": "t-14 t-bold"},
                )
                .find_parent("div", class_="jobs-box__group")
                .find("p", class_="t-14 mb3")
                .text.strip()
            )
        except:
            emp_type = (
                job_container2.find(
                    "h3",
                    text=re.compile("Employment Type"),
                    attrs={"class": "t-14 t-bold"},
                )
                .find_parent("div", class_="jobs-box__group")
                .find("p", class_="t-14 mb3")
                .text.strip()
            )
    except NoSuchElementException:
        print(f"No job employment type found.")
        emp_type = -1
    except errors:
        print(f"Unexpected error getting employment type: {sys.exc_info()[0]}.")
        emp_type = -1
    else:
        print("-" * 20)
        print(f"Employment Type: {emp_type}")
        print("-" * 20)

    # Check if collection successful
    imp_collected = [job_title, job_description, company_name, location]
    if all((var != int(-1)) or (var != str("-1")) for var in imp_collected):
        print("Main data collected successfully.")

    elif any((var == int(-1)) or (var == str("-1")) for var in imp_collected):
        print(
            f"Main data NOT collected successfully. Sleeping for 10 seconds then trying again."
        )
        time.sleep(10)
        try:
            WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable(
                    (By.CSS_SELECTOR, "#main-content > div > section > ul")
                )
            )
            (
                job_title,
                job_description,
                company_name,
                location,
                industry,
                sector,
                level,
                company_url,
                emp_type,
            ) = get_main(driver, soup)
        except:
            print(
                f"Data NOT collected successfully. Not attempting again: {sys.exc_info()[0]}."
            )

    return (
        job_title,
        job_description,
        company_name,
        location,
        industry,
        sector,
        level,
        company_url,
        emp_type,
    )


In [ ]:
# Collect individual job links
def get_each(
    driver,
    main_window,
    window_before,
    keyword,
    site,
    jobs,
    job_soup,
    job_id,
    jobs_count,
    counter,
    page_counter,
    page_number,
    page_number_length,
    save_path,
    json_file_name,
):
    jobs = remove_dupe_dicts(jobs)

    # If results open in new window, ke sure everything loads
    new_window = check_window(driver, main_window, window_before)

    # Get html of page with beautifulsoup4
    html = driver.page_source
    print("Feeding html driver to BeautifulSoup.")
    soup = BeautifulSoup(html, "lxml")

    # try:
    # Get job id, url and age
    print("-" * 20)
    print(f"Job ID: {job_id}")
    print("-" * 20)
    job_url = (
        "https://www.linkedin.com/"
        + job_soup.find(
            "div", class_="full-width artdeco-entity-lockup__title ember-view"
        ).find(
            "a",
            class_="disabled ember-view job-card-container__link job-card-list__title",
        )[
            "href"
        ]
    )
    print(f"Job URL: {job_url}")
    print("-" * 20)
    try:
        job_age_datetime = job_soup.find(
            "li",
            class_=re.compile("^job-card-container__listed-time job-card-container*"),
        ).find("time")
    except:
        print('No date "Job Age" or "Job Date" found.')
        job_age, job_age_number, job_date, job_datetime = assign_all(4, -1)
    else:
        job_age = job_age_datetime.text.strip()
        for s in job_age.split():
            job_age_number = int(s) if s.isdigit() else job_age
        for i in re.findall("[\w +/.]", job_age):
            job_age_unit = str(i) if i.isalpha() else job_age
        job_date = job_age_datetime["datetime"].strip()
        job_datetime = datetime.datetime.strptime(job_date, "%Y-%m-%d")

    print(f"Job Age: {job_age}")
    print("-" * 20)
    print(f"Job Date: {job_date}")
    print("-" * 20)

    #########################################################
    #########################################################
    ####################### TAB INFO #######################
    #########################################################
    #########################################################

    try:
        # Get main data
        (
            job_title,
            job_description,
            company_name,
            location,
            industry,
            sector,
            level,
            company_url,
            emp_type,
        ) = get_main(driver, soup)

        # # In case there is no company tab, -1 for all variables
        # except NoSuchElementException:
        #     print(f'No data collected: {sys.exc_info()[0]}.')
        #     job_title, job_description, company_name, location, industry, sector, level, company_url, emp_type = assign_all(9, -1)
    except:
        print(
            f"Problems with getting comment elements.\nNo data collected: {sys.exc_info()[0]}."
        )
        (
            job_url,
            job_age,
            job_age_number,
            job_date,
            job_title,
            job_description,
            company_name,
            location,
            industry,
            sector,
            level,
            company_url,
            emp_type,
        ) = assign_all(13, -1)

    # Save to dict
    print("Saving to dict...")
    today = datetime.date.today()
    jobs.append(
        {
            "Search Keyword": str(keyword),
            "Platform": site,
            "Job ID": int(job_id),
            "Job Title": str(job_title),
            "Company Name": str(company_name),
            "Location": str(location),
            "Industry": str(industry),
            "Sector": str(sector),
            "Seniority Level": str(level),
            "Job Description": str(job_description),
            "Employment Type": str(emp_type),
            "Company URL": company_url,
            "Job URL": job_url,
            "Job Age": job_age,
            "Job Age Number": job_age_number,
            "Job Date": job_date,
            "Collection Date": (today.strftime("%Y-%m-%d")),
        }
    )

    # Save dict as json file
    jobs = remove_dupe_dicts(jobs)
    print(f"Progress for {keyword} job ads: {len(jobs)} / {int(jobs_count)}")
    print(f"Saving dict to json file as {json_file_name} in location {save_path}")
    with open(save_path + json_file_name, "w") as f:
        json.dump(jobs, f)

    # Check current window and go back if required
    check_window_back(driver, main_window, window_before, new_window)

    return jobs, counter


In [ ]:
def get_page_jobs(
    driver,
    soup,
    keyword,
    keyword_file,
    site,
    main_window,
    window_before,
    counter,
    page_counter,
    page_number,
    page_number_length,
    jobs,
    df_old_jobs,
    save_path,
    json_file_name,
):

    # Get html of page with beautifulsoup4
    html = driver.page_source
    print("Feeding html driver to BeautifulSoup.")
    soup = BeautifulSoup(html, "lxml")
    old_url = driver.title
    # try:
    # WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.CLASS_NAME, 'jobs-search-results display-flex flex-column')))
    try:
        jobs_on_page = soup.find(
            "ul", class_="jobs-search-results__list list-style-none"
        ).find_all(
            "li",
            class_=re.compile("^jobs-search-results__list-item occludable-update*"),
        )
        jobs_count = len(jobs_on_page)
    except AttributeError as e:
        print(
            f"Unexpected error with job links: {e} - {sys.exc_info()[0]}.\nAttempting .children method."
        )
        WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable(
                (
                    By.CLASS_NAME,
                    "jobs-search-results__list-item occludable-update p0 relative ember-view",
                )
            )
        )
        jobs_grid = soup.find_all(
            "ul", class_="jobs-search-results__list list-style-none"
        )
        jobs_on_page = [job.children for job in jobs_grid]
        jobs_count = cardinality.count(jobs_on_page)

    if int(jobs_count) != 0:
        for job_soup in jobs_on_page:
            try:
                last_collected_job_id_soup = int(
                    job_soup.find(
                        "div",
                        class_=re.compile(
                            "^job-card-container relative job-card-list job-card-container--clickable*"
                        ),
                    )["data-job-id"]
                )
            except:
                last_collected_job_id_soup = int(
                    str(job_soup["data-occludable-entity-urn"]).split(":")[-1]
                )
            for job in jobs:
                if int(job["Job ID"]) == last_collected_job_id_soup:
                    last_collected_job_index = int(jobs_on_page.index(job_soup))
                #                     last_collected_job_index = int(index)
                elif int(job["Job ID"]) != last_collected_job_id_soup:
                    last_collected_job_index = 0

        print(f"{int(jobs_count)} job(s) found on page.")
        print(f"Progress for {keyword} job ads: {len(jobs)} / {int(jobs_count)}")

        for job_soup in jobs_on_page:
            try:
                print("-" * 20)
                print(
                    f"JOB NUMBER: {int(jobs_on_page.index(job_soup)) + 1} OUT OF {int(jobs_count)} ON PAGE {int(page_number)}."
                )
                page_counter = page_number - 1
                if page_counter > 0:
                    with open(
                        save_path + f"{site.lower()}_page_counter_{keyword_file}.txt",
                        "w",
                    ) as f:
                        f.write(str(page_counter))
                print("-" * 20)
                job_button = WebDriverWait(driver, 10).until(
                    lambda driver: driver.find_element_by_xpath(xpath_soup(job_soup))
                )
            except:
                # If results open in new window, make sure everything loads
                new_window = check_window(driver, main_window, window_before)
                if (new_window is True) or (old_url != driver.title):
                    # Get jobs
                    jobs, jobs_count, df_jobs, page_counter, counter = get_jobs(
                        keyword.lower(),
                        keyword_url,
                        site,
                        country,
                        driver,
                        counter,
                        page_counter,
                        main_window,
                        df_old_jobs,
                        jobs,
                        save_path.lower(),
                        json_file_name.lower(),
                        df_file_name.lower(),
                    )
                elif (new_window is False) or (old_url == driver.title):
                    print(
                        f"Problem getting job soup for {str(keyword)}: {sys.exc_info()[0]}. Trying againg."
                    )
                    last_job_soup_xpath = xpath_soup(
                        jobs_on_page[int(jobs_on_page.index(job_soup)) - 1]
                    )
                    last_job_soup = WebDriverWait(driver, 10).until(
                        lambda driver: driver.find_element_by_xpath(last_job_soup_xpath)
                    )
                    job_soup = WebDriverWait(driver, 10).until(
                        lambda last_job_soup: last_job_soup.find_element_by_xpath(
                            "following-sibling::*[1]"
                        )
                    )
                    job_xpath = xpath_soup(job_soup)
                    job_button = WebDriverWait(driver, 10).until(
                        lambda driver: driver.find_element_by_xpath(job_xpath)
                    )

            # If results open in new window, make sure everything loads
            new_window = check_window(driver, main_window, window_before)
            if (new_window is False) or (old_url == driver.title):
                try:
                    job_button.click()
                except:
                    print(
                        f"Click did not work. Trying again to get job soup for {str(keyword)}."
                    )
                    # If results open in new window, make sure everything loads
                    new_window = check_window(driver, main_window, window_before)
                    if (new_window is True) or (old_url != driver.title):
                        # Get jobs
                        jobs, jobs_count, df_jobs, page_counter, counter = get_jobs(
                            keyword.lower(),
                            keyword_url,
                            site,
                            country,
                            driver,
                            counter,
                            page_counter,
                            main_window,
                            df_old_jobs,
                            jobs,
                            save_path.lower(),
                            json_file_name.lower(),
                            df_file_name.lower(),
                        )
                    elif (new_window is False) or (old_url == driver.title):
                        print(
                            f"Problem clicking job button for {str(keyword)}: {sys.exc_info()[0]}. Trying againg."
                        )
                        job_button.click()

                # Get job id
                try:
                    job_id = job_soup.find(
                        "div",
                        class_=re.compile(
                            "^job-card-container relative job-card-list job-card-container--clickable*"
                        ),
                    )["data-job-id"]
                except TypeError:
                    job_id = int(
                        str(job_soup["data-occludable-entity-urn"]).split(":")[-1]
                    )

                # Check if already collected
                jobs, job_present = id_check(
                    driver,
                    main_window,
                    window_before,
                    keyword,
                    site,
                    df_old_jobs,
                    jobs,
                    job_soup,
                    job_id,
                    jobs_count,
                    save_path,
                    json_file_name,
                )
                if job_present is False:
                    if int(jobs_on_page.index(job_soup)) != int(jobs_count):
                        jobs, counter = get_each(
                            driver,
                            main_window,
                            window_before,
                            keyword,
                            site,
                            jobs,
                            job_soup,
                            job_id,
                            jobs_count,
                            counter,
                            page_counter,
                            page_number,
                            page_number_length,
                            save_path,
                            json_file_name,
                        )
                    elif int(jobs_on_page.index(job_soup)) == int(jobs_count):
                        pass
                elif job_present is True:
                    pass

                # If results open in new window, make sure everything loads
                new_window = check_window(driver, main_window, window_before)
            elif (new_window is True) or (old_url != driver.title):
                driver.back()
                jobs, jobs_count, counter = get_page_jobs(
                    driver,
                    soup,
                    keyword,
                    keyword_file,
                    site,
                    main_window,
                    window_before,
                    counter,
                    page_counter,
                    page_number,
                    page_number_length,
                    jobs,
                    df_old_jobs,
                    save_path,
                    json_file_name,
                )
        # Scroll down
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")

    elif int(jobs_count) == 0:
        print(
            f"No jobs results found for {str(keyword)}. No error detected. Moving on to next search."
        )
    #     except:
    #         print(f'No jobs results found for {str(keyword)}: {sys.exc_info()[0]}. Moving on to next search.')
    #         job_id, job_url, job_age, job_age_number, job_date, job_title, job_description, company_name, location, industry, sector, level, company_url, emp_type = assign_all(14, -1)

    # Save dict as json file
    jobs = remove_dupe_dicts(jobs)
    try:
        print(f"Progress for {keyword} job ads: {len(jobs)} / {int(jobs_count)}")
    except UnboundLocalError:
        jobs_count = len(jobs)
        print(f"Progress for {keyword} job ads: {len(jobs)} / {int(jobs_count)}")
    print(f"Saving dict to json file as {json_file_name} in location {save_path}")
    with open(save_path + json_file_name, "w") as f:
        json.dump(jobs, f)

    return jobs, jobs_count, counter


### Get-jobs function

In [ ]:
# MAIN FUNCTION TO GET JOB ADS
def get_jobs(
    driver,
    keyword,
    keyword_url,
    keyword_file,
    site,
    country,
    counter,
    page_counter,
    main_window,
    df_old_jobs,
    jobs,
    save_path,
    json_file_name,
    df_file_name,
):

    url = f"https://www.linkedin.com/jobs/search?keywords={keyword_url}&location={country}"

    # Go to page and establish main windows
    print(f"Going to url: {url}")
    driver.get(url)
    window_before = driver.window_handles[0]
    print(f"First Window: {window_before}")
    print(f"Current page title: {driver.title}")
    pre_signup_driver_title = driver.title

    # Get html of page with beautifulsoup4
    html = driver.page_source
    print("Feeding html driver to BeautifulSoup.")
    soup = BeautifulSoup(html, "lxml")

    try:
        signup_form = WebDriverWait(driver, 10).until(
            lambda driver: driver.find_element_by_class_name(
                "join-form__form-body-agreement"
            )
        )
        if (signup_form.is_enabled()) or (
            "By clicking Agree & Join, you agree to the LinkedIn" in signup_form.text
        ):
            signin_button_general = WebDriverWait(driver, 10).until(
                lambda driver: driver.find_element_by_class_name("form-toggle")
            )
            if (signin_button_general.is_enabled()) or (
                "Sign in" in signin_button_general.text
            ):
                try:
                    signin_button_general.click()
                except (
                    ElementNotInteractableException,
                    ElementClickInterceptedException,
                ):
                    time.sleep(1)
                    signin_button_general.click()

                try:
                    sign_in(driver)
                except (
                    ElementNotInteractableException,
                    ElementClickInterceptedException,
                ):
                    time.sleep(1)
                    sign_in(driver)
                print("Signed in.")
    except TimeoutException:
        signin_button = WebDriverWait(driver, 10).until(
            lambda driver: driver.find_element_by_class_name("nav__button-secondary")
        )
        if (signin_button.is_enabled()) or ("Sign in" in signin_button.text):
            try:
                signin_button.click()
            except (ElementNotInteractableException, ElementClickInterceptedException):
                time.sleep(1)
                signin_button.click()
            print("Signing in.")

            try:
                sign_in(driver)
            except (ElementNotInteractableException, ElementClickInterceptedException):
                time.sleep(1)
                sign_in(driver)
            print("Signed in.")
    #     except:
    #         print('Already signed in.')

    url = f"https://www.linkedin.com/jobs/search?keywords={keyword_url}&location={country}"

    # Go to page and establish main windows
    print(f"Going to url: {url}")
    driver.get(url)
    window_before = driver.window_handles[0]
    print(f"First Window: {window_before}")
    print(f"Current page title: {driver.title}")
    pre_signup_driver_title = driver.title

    if "Security Verification" in str(driver.title):
        print("!" * 30, "CAPTCHA FOUND", "!" * 30)
        act_cool(5, 500)
        ! openpyn nl
        ! funckymonkey89
        # Get jobs
        jobs, jobs_count, df_jobs, page_counter, counter = get_jobs(
            driver,
            keyword.lower(),
            keyword_url,
            keyword_file,
            site,
            country,
            counter,
            page_counter,
            main_window,
            df_old_jobs,
            jobs,
            save_path.lower(),
            json_file_name.lower(),
            df_file_name.lower(),
        )

    # Find page number
    try:
        page_number_length, page_number = find_page_number(
            driver, soup, page_counter, keyword
        )
    except errors:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
        time.sleep(2)
        try:
            try:
                load_more_xpath = xpath_soup(
                    soup.find(
                        "section",
                        class_="m3",
                        attrs={"aria-label": "Load more results"},
                    )
                    .find("div", class_="jobs-search-two-pane__load-more")
                    .find("button")
                )
                load_more_button = WebDriverWait(driver, 10).until(
                    lambda driver: driver.find_element_by_xpath(load_more_xpath)
                )
            except:
                load_more_button = WebDriverWait(driver, 10).until(
                    lambda driver: driver.find_element_by_xpath('//*[@id="ember1142"]')
                )
        except AttributeError:
            pass
        else:
            if (load_more_button.is_displayed()) and (load_more_button.is_enabled()):
                page_number = page_counter
                page_number_length = 1
                print('"Load More" button found so no next button or page number.')
            elif (not load_more_button.is_displayed()) and (
                not load_more_button.is_enabled()
            ):
                page_number = page_counter
                page_number_length = 1
                print(
                    'Could not get page number. No "Load More" button or next button found.'
                )
    except:
        pass

    # Check for errors
    # try:
    not_page_error = WebDriverWait(driver, 10).until_not(
        lambda driver: driver.find_element_by_class_name(
            re.compile("^jobs-search-no-results__image")
        )
    )

    while not_page_error:
        # Find jobs in page
        print("Finding job links in page.")
        jobs, jobs_count, counter = get_page_jobs(
            driver,
            soup,
            keyword,
            keyword_file,
            site,
            main_window,
            window_before,
            counter,
            page_counter,
            page_number,
            page_number_length,
            jobs,
            df_old_jobs,
            save_path,
            json_file_name,
        )

        # Get html of page with beautifulsoup4
        html = driver.page_source
        print("Feeding html driver to BeautifulSoup.")
        soup = BeautifulSoup(html, "lxml")

        try:
            not_end_of_load = WebDriverWait(driver, 10).until_not(
                lambda driver: driver.find_element_by_class_name(
                    "query-expansion-suggestions"
                )
            )
        except:
            not_end_of_load = WebDriverWait(driver, 10).until_not(
                lambda driver: driver.find_element_by_class_name(
                    "occludable-update ember-view"
                )
            )
        if not_end_of_load:
            new_driver_title = driver.title
            if new_driver_title == driver.title:
                # Find next jobs / page
                try:
                    try:
                        load_more(
                            driver,
                            soup,
                            keyword,
                            keyword_file,
                            site,
                            main_window,
                            window_before,
                            counter,
                            page_counter,
                            page_number,
                            page_number_length,
                            jobs,
                            jobs_count,
                            df_old_jobs,
                            save_path,
                            json_file_name,
                        )
                        # Get jobs
                        jobs, jobs_count, counter = get_page_jobs(
                            driver,
                            soup,
                            keyword,
                            keyword_file,
                            site,
                            main_window,
                            window_before,
                            counter,
                            page_counter,
                            page_number,
                            page_number_length,
                            jobs,
                            df_old_jobs,
                            save_path,
                            json_file_name,
                        )
                    except (AttributeError, TimeoutException):
                        page_number_length, page_number = find_page_number(
                            driver, soup, page_counter, keyword
                        )
                        if int(page_number_length) == int(page_number):
                            print(
                                f"Last page reached at page {page_number}. End of data collection for {keyword}.\nGot {int(jobs_count)} jobs."
                            )
                            break
                        elif int(page_number_length) != int(page_number):
                            page_current = soup.find(
                                "li",
                                class_="artdeco-pagination__indicator artdeco-pagination__indicator--number active selected ember-view",
                            ).find("button", {"aria-current": "true"})
                            next_page_button_soup = page_current.find_next_siblings(
                                "li",
                                class_="artdeco-pagination__indicator artdeco-pagination__indicator--number ember-view",
                            ).find("button")
                            next_page_button_xpath = xpath_soup(next_page_button_soup)
                            next_page_button.click()
                            print(
                                f"Going to next page {int(page_number)+1}.\nGot {int(jobs_count)} jobs."
                            )
                        elif int(page_number_length) == -1:
                            load_more(
                                driver,
                                soup,
                                keyword,
                                keyword_file,
                                site,
                                main_window,
                                window_before,
                                counter,
                                page_counter,
                                page_number,
                                page_number_length,
                                jobs,
                                jobs_count,
                                df_old_jobs,
                                save_path,
                                json_file_name,
                            )
                            # Get jobs
                            jobs, jobs_count, counter = get_page_jobs(
                                driver,
                                soup,
                                keyword,
                                keyword_file,
                                site,
                                main_window,
                                window_before,
                                counter,
                                page_counter,
                                page_number,
                                page_number_length,
                                jobs,
                                df_old_jobs,
                                save_path,
                                json_file_name,
                            )
                    except:
                        # Get html of page with beautifulsoup4
                        html = driver.page_source
                        print("Feeding html driver to BeautifulSoup.")
                        soup = BeautifulSoup(html, "lxml")
                        next_page_xpath = xpath_soup(
                            soup.find("a", attrs={"data-test": "pagination-next"})
                        )
                        next_page_button = WebDriverWait(driver, 10).until_not(
                            lambda driver: driver.find_element_by_xpath(next_page_xpath)
                        )
                        if int(page_number_length) == int(page_number):
                            print(
                                f"Last page reached at page {page_number}. End of data collection for {keyword}.\nGot {int(jobs_count)} jobs."
                            )
                            break
                    try:
                        next_page_button.click()
                        print(
                            f"Going to next page {int(page_number)+1}.\nGot {int(jobs_count)} jobs."
                        )
                        # Get jobs
                        jobs, jobs_count, counter = get_page_jobs(
                            driver,
                            soup,
                            keyword,
                            keyword_file,
                            site,
                            main_window,
                            window_before,
                            counter,
                            page_counter,
                            page_number,
                            page_number_length,
                            jobs,
                            df_old_jobs,
                            save_path,
                            json_file_name,
                        )
                    except TimeoutException:
                        load_more(
                            driver,
                            soup,
                            keyword,
                            keyword_file,
                            site,
                            main_window,
                            window_before,
                            counter,
                            page_counter,
                            page_number,
                            page_number_length,
                            jobs,
                            jobs_count,
                            df_old_jobs,
                            save_path,
                            json_file_name,
                        )
                        # Get jobs
                        jobs, jobs_count, counter = get_page_jobs(
                            driver,
                            soup,
                            keyword,
                            keyword_file,
                            site,
                            main_window,
                            window_before,
                            counter,
                            page_counter,
                            page_number,
                            page_number_length,
                            jobs,
                            df_old_jobs,
                            save_path,
                            json_file_name,
                        )

                except:
                    if new_driver_title == driver.title:
                        load_more(
                            driver,
                            soup,
                            keyword,
                            keyword_file,
                            site,
                            main_window,
                            window_before,
                            counter,
                            page_counter,
                            page_number,
                            page_number_length,
                            jobs,
                            jobs_count,
                            df_old_jobs,
                            save_path,
                            json_file_name,
                        )
                        # Get jobs
                        jobs, jobs_count, counter = get_page_jobs(
                            driver,
                            soup,
                            keyword,
                            keyword_file,
                            site,
                            main_window,
                            window_before,
                            counter,
                            page_counter,
                            page_number,
                            page_number_length,
                            jobs,
                            df_old_jobs,
                            save_path,
                            json_file_name,
                        )
                    elif new_driver_title != driver.title:
                        driver.back()
                        time.sleep(2)
                        load_more(
                            driver,
                            soup,
                            keyword,
                            keyword_file,
                            site,
                            main_window,
                            window_before,
                            counter,
                            page_counter,
                            page_number,
                            page_number_length,
                            jobs,
                            jobs_count,
                            df_old_jobs,
                            save_path,
                            json_file_name,
                        )
                        # Get jobs
                        jobs, jobs_count, counter = get_page_jobs(
                            driver,
                            soup,
                            keyword,
                            keyword_file,
                            site,
                            main_window,
                            window_before,
                            counter,
                            page_counter,
                            page_number,
                            page_number_length,
                            jobs,
                            df_old_jobs,
                            save_path,
                            json_file_name,
                        )
            elif new_driver_title != driver.title:
                driver.back()
                time.sleep(2)
                load_more(
                    driver,
                    soup,
                    keyword,
                    keyword_file,
                    site,
                    main_window,
                    window_before,
                    counter,
                    page_counter,
                    page_number,
                    page_number_length,
                    jobs,
                    jobs_count,
                    df_old_jobs,
                    save_path,
                    json_file_name,
                )
                # Get jobs
                jobs, jobs_count, counter = get_page_jobs(
                    driver,
                    soup,
                    keyword,
                    keyword_file,
                    site,
                    main_window,
                    window_before,
                    counter,
                    page_counter,
                    page_number,
                    page_number_length,
                    jobs,
                    df_old_jobs,
                    save_path,
                    json_file_name,
                )

        elif not not_end_of_load:
            print('No jobs on page. Showing page error "not_end_of_load".')
            break

        print("-" * 20)
        print(f"REACHED END OF JOBS.")
        print("-" * 20)
    else:
        print('No jobs on page. Showing page error "not_page_error".')
        jobs_count = len(jobs)
        print(f"No job results found for {str(keyword)}. Moving on to next search.")
    # except:
    #     jobs_count = len(jobs)
    #     print(f'No jobs results found for {str(keyword)}. Moving on to next search.')

    ############################################### FINAL SAVING AND CLEANUP ###############################################

    # Save dict as json file
    print("Final save.")
    jobs = remove_dupe_dicts(jobs)
    try:
        print(f"Progress for {keyword} job ads: {len(jobs)} / {int(jobs_count)}")
    except:
        print(f"Progress for {keyword} job ads: {len(jobs)}")
    print(f"Saving dict to json file as {json_file_name} in location {save_path}")
    with open(save_path + json_file_name, "w") as f:
        json.dump(jobs, f)

    # Save jobs to df
    print(f"Saving {keyword} jobs data to df...")
    if jobs:
        df_jobs = pd.DataFrame(jobs)
    elif not jobs:
        print(f"Jobs dict for {keyword} is empty.")
        df_jobs = pd.DataFrame()

    # Save df as csv
    if not df_jobs.empty:
        df_jobs.append(df_old_jobs, ignore_index=True)
        df_jobs = clean_df(df_jobs)
        df_jobs = save_df(
            keyword, df_jobs, save_path, keyword_file.lower(), df_file_name.lower()
        )
    elif df_jobs.empty:
        df_jobs = df_old_jobs

    return jobs, jobs_count, df_jobs, page_counter, counter


### Run get_jobs function

In [ ]:
# if __name__ == '__main__':
def run_all_linkedin(
    site="LinkedIn",
    language="English",
    country="Netherlands",
    incognito_enabled=True,
    headless_enabled=False,
    proxy_enabled=False,
    logging_enabled=False,
    print_enabled=True,
):

    # Get keywords
    (
        keywords_list,
        keywords_sector,
        keywords_womenvocc,
        keywords_menvocc,
        keywords_genvsect,
        keywords_oldvocc,
        keywords_youngvocc,
        keywords_agevsect,
    ) = get_keyword_list(print_enabled=print_enabled)
    # remove_list = ['software', 'service', 'storage', 'information', 'sales', 'culture']
    # for k in remove_list:
    #     keywords_list.remove(k)

    # Start driver
    driver = get_driver(select_driver)

    # Start for-loop
    keywords_oldvocc = ["industry"]
    for keyword in keywords_youngvocc:
        counter = 1
        page_counter = 0
        recollect = False

        # Check internet connection
        if not is_connected(driver):
            is_connected(driver)

        # Get main info
        (
            keyword_url,
            keyword_file,
            save_path,
            json_file_name,
            df_file_name,
            logs_file_name,
            filemode,
        ) = main_info(keyword, site)

        # Set up log
        if logging_enabled is True:
            print("Logging enabled.")
            writer = MyWriter(logs_file_name.lower(), filemode)
            sys.stdout = writer
            logger = logging.getLogger(__name__)
            logging.basicConfig(
                filename=writer.LOGS_PATH + logs_file_name.lower(),
                filemode=filemode,
                level=logging.CRITICAL,
                format="%(asctime)s:%(levelname)s: %(message)s",
            )
        elif logging_enabled is False:
            print("No logging enabled.")

        # Load and merge existing dict and df
        print(f"All files will be saved in folder: {save_path}{json_file_name}")
        jobs, df_old_jobs = load_merge_dict_df(
            keyword, save_path, df_file_name, json_file_name
        )

        # Beging data collection
        print(
            "-" * 20,
            f"BEGINING OF DATA COLLECTION FOR {keyword.upper()} JOB ADS",
            "-" * 20,
        )
        main_window = driver.current_window_handle
        print(f"Main Window: {main_window}")

        # Load and save page_counter
        page_counter_path = (
            save_path + f"{site.lower()}_page_counter_{keyword_file}.txt"
            if recollect is False
            else save_path + f"{site.lower()}_page_counter_done_{keyword_file}.txt"
        )

        if is_non_zero_file(page_counter_path) is True:
            with open(page_counter_path) as f:
                page_counter = int(f.read())
        elif is_non_zero_file(page_counter_path) is False:
            page_counter = 1
        with open(page_counter_path, "w") as f:
            f.write(str(page_counter))

        print(f"Starting collection from page number: {page_counter}")

        # Start threads
        popup_thread(driver, popup, popup_checker)
        # stale_element_thread(driver, stale_element, stale_element_checker)
        # act_cool_thread(driver, act_cool, act_cool_checker)

        # Get jobs
        jobs, jobs_count, df_jobs, page_counter, counter = get_jobs(
            driver,
            keyword.lower(),
            keyword_url,
            keyword_file,
            site,
            country,
            counter,
            page_counter,
            main_window,
            df_old_jobs,
            jobs,
            save_path.lower(),
            json_file_name.lower(),
            df_file_name.lower(),
        )

        # Save df as csv
        df_jobs = save_df(
            keyword, df_jobs, save_path, keyword_file.lower(), df_file_name.lower()
        )
        print(
            "-" * 20, f"END OF DATA COLLECTION FOR {keyword.upper()} JOB ADS", "-" * 20
        )

    # Close and quit driver
    print(f"Closing driver.")
    driver.close()
    driver.quit()
    # thread_popup_checker.terminate()
    # thread_stale_element_checker.terminate()
    # thread_act_cool_checker.terminate()
    # sys.stdout.close()
    # sys.exit()

    return df_jobs


In [ ]:
df_jobs_linkedin = run_all_linkedin()


In [ ]:
# keyword = 'network specialist'
# site = 'LinkedIn'
# keyword_from_list = True
# site_from_list = False
# df_jobs = post_cleanup(keyword, site)
